## Imports

In [1]:
import pandas as pd
import math
from IPython.display import SVG, display
import numpy as np
import colorsys
from math import gcd
from matplotlib import colormaps

## Functions needed

In [2]:
def is_prime(num):
    """
    Function to check if a number is prime.
    """
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True

def SophieGermainNumber(n):
    """
    Gets all Sophie Germain primes below n.
    """
    sophie_germain_primes = []
    
    for i in range(2, n):
        if is_prime(i) and is_prime(2 * i + 1):
            sophie_germain_primes.append(i)
    
    return sophie_germain_primes

def find_shortest_cycle(list):
    """
    Function that takes repeating sequences in a given list, and returns the smallest cycle.
    For example the list [1,2,1,2,1,2] should be turned into [1,2].
    From arithmetic we know that each value can only come up once in the cycle, 
    so if the first value of the list repeats, we can already cut off the list there.
    """
    first = list[0]
    result = [first]
    for num in list[1:]:
        if num == first:
            break
        result.append(num)
    return result

def full_row(p,m):
    """
    Warning! Check yourself whether the selected p is a Sophie Germain prime.
    This is not going to matter later on, because it will only select from a list of SG primes.
    """
    list_1 = []
    n_max = m
    
    for n in range(1, n_max+1): 
        a = pow(p, n, m)
        list_1.append(a)

    shortest_cycle = find_shortest_cycle(list_1)
            
    return [p,m,shortest_cycle,len(shortest_cycle)]

def df_maker(p,m):
    """
    Making a df with a constant m, an n set to m because it shouldn't be able to exceed m. We'll do all SG primes below a certain number sg.
    gcd checks if p and m are relatively prime or not.
    """

    data=[]
    
    for p in SophieGermainNumber(p+1):
        if gcd(p, m) == 1:
            data.append(full_row(p,m))
        else:
            continue
    
    df = pd.DataFrame(data, columns=['p', 'm', 'p^n mod m', 'cycle length'])
    return df

def seq_incr(seq):
    """
    Checks if the values from the start only increase.
    """
    return all(seq[i] < seq[i+1] for i in range(len(seq) - 1))

def seq_odd(seq):
    """
    Checks if the sequence is odd.
    """
    return all(x % 2 == 1 for x in seq)

## Final animation

In [91]:
#for the animation I do want to do global scaling

def svg_radial_plot_maker_roman_global(dfs):
    """
    Makes an animation of multiple radial plots, one per each dataframe created by the pre-defined df_maker.
    """
    ################################### DRAW ONCE ############################################
    
    #storing all elements in here
    svg_elements = []

    #I hardcoded these
    width = 600 #700
    height = 600 #700

    #places the center of the axis in the center of the image
    center_x = width / 2
    center_y = height / 2

    #background color
    svg_elements.append(f'<rect width="{width}" height="{height}" fill="#e0ecf4" />')

    #cycle lengths over all dataframes
    cycle_lengths_dfs = [l for df in dfs for l in df["cycle length"]]

    #need that over all dataframes in this bit because otherwise the scaling doesn't work
    max_cycle_length_dfs = max(cycle_lengths_dfs)
    max_radius = min(width, height) / 2 * 0.9  #largest circle radius is just a little smaller than the width or height
    scaling_factor = max_radius / max_cycle_length_dfs #basically the scale at which each circle is drawn is based on how small a factor it is of max 

    #add here all the magnitudes in one list, excluding the sequences of length 1 because those cannot be computed the magnitude of
    magnitudes = []
    for df in dfs:
        for sequence in df['p^n mod m']:
            if len(sequence) >= 2:
                dft_values = np.fft.fft(sequence)
                harmonic = 1
                magnitude = np.abs(dft_values[harmonic])
                magnitudes.append(magnitude)

    #define the largest magnitude here already so that it can be normalized later on
    max_value_magnitude = max(magnitudes) #this is roughly 44
    min_value_magnitude = min(magnitudes) #checked and should be 1

    #we're going to make here a list of all Germain primes within primes p, hardcoded again sorry
    sg_set = set(SophieGermainNumber(72504))

    ##########################################################################################


    #for loop that considers each dataframe in the dataframes given as input
    for another_index, df in enumerate(dfs):
        m_val = int(df["m"].unique()[0])

        group = [f'<g id="plot_{another_index}" opacity="0">']

        group.append(
            f'<animate attributeName="opacity" values="0;1;0" keyTimes="0;0.25;1" '
            f'dur="6s" begin="{another_index*1.5}s" fill="freeze" repeatCount="1" calcMode="spline" keySplines=".7 0 .9 1; .1 0 .5 1" />'
        ) #changed linear calcmode to spline for a more ripple effect, messed around with duration and begin

        #definitions of attributes per dataframe
        no_of_rows = len(df)
        data_points = df["cycle length"].tolist()
        angle_step = 360 / no_of_rows
        max_cycle_length_for_m = df["cycle length"].max()

        #index over each point of data in a dataframe
        for index, data_point in enumerate(data_points):
            radius = data_point * scaling_factor
            angle_deg = -index * angle_step
            angle_rad = math.radians(angle_deg)

            x = center_x + radius * math.cos(angle_rad)
            y = center_y + radius * math.sin(angle_rad)

            
            ############################## COLOR TRANSFORM ###################################
            
            seq = df["p^n mod m"].iloc[index]

            if len(seq) >= 2:
                dft_values = np.fft.fft(seq)
                harmonic = 1
                magnitude = np.abs(dft_values[harmonic])
                normalize = (magnitude-min_value_magnitude) / (max_value_magnitude-min_value_magnitude)

                #blues colormapping, trying this instead of HSL because it is fast to implement and is a standard colormap
                #this means that white indicates a magnitude close to one
                #and darker means high magnitudes towards 44
                cmap = colormaps['Blues']
                r, g, b, _ = cmap(normalize)
                
                fill_color = f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}'


            else:
                fill_color = "#cccccc" #gray as default for the cycles of length 1

            ##################################################################################

            ############################## SHAPE TRANSFORM ###################################

            if data_point == max_cycle_length_for_m: #instead of elif
                group.append(f'<circle cx="{x:.2f}" cy="{y:.2f}" r="9" fill="{fill_color}" stroke="#327bb5" stroke-width="0.3" />')
                
            else:
                group.append(f'<circle cx="{x:.2f}" cy="{y:.2f}" r="5" fill="{fill_color}" stroke="#327bb5" stroke-width="0.3" />')

            if any(val in sg_set for val in seq):
                    group.append(f'''<g transform="translate({x:.2f},{y:.2f}) scale(0.9)">
                        <path d="M 0,0 L 10,0 A 10,10 0 1,1 -9.51,3.09 Z"
                              fill="green" stroke="darkgreen" stroke-width="1"/>
                    </g>''')
            
            if seq_odd(seq):
                flower = '''<g>
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(0) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(72) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(144) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(216) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(288) translate(0, -4)" />
                    <circle cx="0" cy="0" r="1.5" fill="pink" />
                '''
            
                if seq_incr(seq):
                    flower += '<circle cx="0" cy="0" r="1.5" fill="yellow" />'
            
                flower += '</g>'
      
                group.append(f'''<g transform="translate({x:.2f},{y:.2f}) scale(0.9)">
                    {flower}
                </g>''')
                
            ##################################################################################

        group.append('</g>')
        svg_elements.append("\n".join(group))
    
    svg_content = "\n".join(svg_elements)
    svg_output = f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">\n{svg_content}\n</svg>'
    
    return svg_output

In [92]:
dfs = [df_maker(3863,2),
       df_maker(3863,3),
       df_maker(10613,5),
       df_maker(10613,7), 
       df_maker(10613,11),
       df_maker(10613,13),
       df_maker(18041,17),
       df_maker(18041,19)]
         
svg_code = svg_radial_plot_maker_roman_global(dfs)
with open("radial_plot_10.svg", "w") as f:
    f.write(svg_code)

## Final set of frames

In [93]:

def svg_radial_frames_maker_roman_local(dfs):
    """
    Makes separate frames.
    """
    ################################### DRAW ONCE ############################################
    #frames instead of elements
    frames = []

    #I hardcoded these
    width = 600
    height = 600

    #places the center of the axis in the center of the image
    center_x = width / 2
    center_y = height / 2

    #removed bg here

    #cycle lengths over all dataframes
    cycle_lengths_dfs = [l for df in dfs for l in df["cycle length"]]

    #max_cycle_length_dfs = max(cycle_lengths_dfs)
    max_radius = min(width, height) / 2 * 0.9  #largest circle radius is just a little smaller than the width or height
    #scaling_factor = max_radius / max_cycle_length_dfs #basically the scale at which each circle is drawn is based on how small a factor it is of max 

    #add here all the magnitudes in one list, excluding the sequences of length 1 because those cannot be computed the magnitude of
    magnitudes = []
    for df in dfs:
        for sequence in df['p^n mod m']:
            if len(sequence) >= 2:
                dft_values = np.fft.fft(sequence)
                harmonic = 1
                magnitude = np.abs(dft_values[harmonic])
                magnitudes.append(magnitude)

    #define the largest magnitude here already so that it can be normalized later on
    max_value_magnitude = max(magnitudes) #this is roughly 44
    min_value_magnitude = min(magnitudes) #checked and should be 1

    sg_set = set(SophieGermainNumber(72504))

    ##########################################################################################

    #for loop that considers each dataframe in the dataframes given as input
    for another_index, df in enumerate(dfs):
        #removed this stuff that was here because it was about the animation and we don't need that now
        #also intialize elements inside the loop here, as well as setting the background again each time
        svg_elements = []
        svg_elements.append(f'<rect width="{width}" height="{height}" fill="#e0ecf4" />') #background
        
        #definitions of attributes per dataframe
        no_of_rows = len(df)
        data_points = df["cycle length"].tolist()
        angle_step = 360 / no_of_rows
        max_cycle_length_for_m = df["cycle length"].max()
        scaling_factor = max_radius / max_cycle_length_for_m

        #index over each point of data in a dataframe
        for index, data_point in enumerate(data_points):
            radius = data_point * scaling_factor
            angle_deg = -index * angle_step
            angle_rad = math.radians(angle_deg)

            x = center_x + radius * math.cos(angle_rad)
            y = center_y + radius * math.sin(angle_rad)


            ############################## COLOR TRANSFORM ###################################
            
            seq = df["p^n mod m"].iloc[index]

            if len(seq) >= 2:
                dft_values = np.fft.fft(seq)
                harmonic = 1
                magnitude = np.abs(dft_values[harmonic])
                normalize = (magnitude-min_value_magnitude) / (max_value_magnitude-min_value_magnitude)

                #blues colormapping, trying this instead of HSL because it is fast to implement and is a standard colormap
                #this means that white indicates a magnitude close to one
                #and darker means high magnitudes towards 44
                cmap = colormaps['Blues']
                r, g, b, _ = cmap(normalize)
                
                fill_color = f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}'


            else:
                fill_color = "#cccccc" #gray as default for the cycles of length 1

            ##################################################################################


            ############################## SHAPE TRANSFORM ###################################
            
            if data_point == max_cycle_length_for_m:
                svg_elements.append(f'<circle cx="{x:.2f}" cy="{y:.2f}" r="9" fill="{fill_color}" stroke="#327bb5" stroke-width="0.3" />')
                
            else:
                svg_elements.append(f'<circle cx="{x:.2f}" cy="{y:.2f}" r="5" fill="{fill_color}" stroke="#327bb5" stroke-width="0.3" />')
            
            if any(val in sg_set for val in seq):
                svg_elements.append(f'''<g transform="translate({x:.2f},{y:.2f}) scale(0.9)">
                    <path d="M 0,0 L 10,0 A 10,10 0 1,1 -9.51,3.09 Z"
                          fill="green" stroke="darkgreen" stroke-width="1"/>
                </g>''')
            
            if seq_odd(seq):
                flower = '''<g>
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(0) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(72) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(144) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(216) translate(0, -4)" />
                    <ellipse rx="2" ry="5" fill="pink" transform="rotate(288) translate(0, -4)" />
                    <circle cx="0" cy="0" r="1.5" fill="pink" />
                '''
            
                if seq_incr(seq):
                    flower += '<circle cx="0" cy="0" r="1.5" fill="yellow" />'
            
                flower += '</g>'
      
                svg_elements.append(f'''<g transform="translate({x:.2f},{y:.2f}) scale(0.9)">
                    {flower}
                </g>''')
    
            ##################################################################################

        svg_content = "\n".join(svg_elements)
        svg_output = f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">\n{svg_content}\n</svg>'
        frames.append(svg_output)

    return frames

In [94]:
frames = svg_radial_frames_maker_roman_local(dfs)

for i, frame in enumerate(frames):
    with open(f"frame_{i+1}.svg", "w") as f:
        f.write(frame)